### Import libraries and load env variables

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv

load_dotenv()

True

### Load data

In [2]:
# Define paths
# fpath_data = os.environ["FPATH_DATA"]
fpath_data = os.environ.get("FPATH_DATA")
fpath_dicts = os.environ.get("FPATH_DICTS")

# Load order items data
order_items = pd.read_csv(fpath_data+"order_items.csv")

# Show sample data
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [3]:
# Inspect order_items data
order_items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


In [4]:
# No nulls in the df
order_items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [5]:
# Check datatypes
order_items.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [6]:
# Convert shipping_limit_date to timestamps
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])
order_items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [7]:
# Lets load data regarding product category, drop redundant columns, check the data, and then join the data to order_items
product_categories = pd.read_csv(fpath_data+"products.csv")
product_categories = product_categories[["product_id", "product_category_name"]]

# Turns out 610 product id's doesnt have any category at all, I drop those rows
product_categories['product_category_name'].isnull().sum()

610

In [8]:
# Drop nulls
product_categories = product_categories.dropna()
# Double check for nulls
product_categories.isnull().sum()

product_id               0
product_category_name    0
dtype: int64

In [9]:
# Map categories to english ones
categories_translation_data = pd.read_csv(fpath_data+"product_category_name_translation.csv")
categories_translation_data.isnull().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [10]:
product_categories = pd.merge(product_categories, categories_translation_data, how='left', on='product_category_name')
product_categories.isnull().sum()

product_id                        0
product_category_name             0
product_category_name_english    13
dtype: int64

In [11]:
# Instead of dropping 13 rows w/o translation, use the same data as originally
# product_categories[product_categories['product_category_name_english'].isnull()] # prior df check
product_categories['product_category_name_english'] = product_categories['product_category_name_english'].fillna(product_categories['product_category_name'])

In [12]:
# Join product category to orders
order_items = pd.merge(order_items, product_categories, how='left', on='product_id')
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,garden_tools
...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,utilidades_domesticas,housewares
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,informatica_acessorios,computers_accessories
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,esporte_lazer,sports_leisure
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,informatica_acessorios,computers_accessories


In [13]:
# Check again for null products after the join - turns out 1603 are without a category
order_items.isnull().sum()

order_id                            0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
product_category_name            1603
product_category_name_english    1603
dtype: int64

In [14]:
# I decided to drop those without any category prescribed as those might be incorrect data
order_items = order_items.dropna(subset=['product_category_name'])

### Basic EDA

In [15]:
# What is the shape of our data?
order_items.shape

(111047, 9)

In [19]:
# How many categories of products are out there?
order_items['product_category_name'].nunique()

73